In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
categories = ['banana','beach','car','church','lighthouse','mountain','mug','plane']
path='../Analysis/human_rating_bycat'

In [3]:
for i, cate in enumerate(categories):
    col = [cate+'_'+str(i+1) for i in range(16)]
    new = pd.read_csv(path+'/'+cate+'.csv')
    new.columns = col
    if i == 0:
        old = new
    else:
        old = pd.concat([old,new],axis=1)
df = old

(Test) Use quantiles to remove outliers 

In [4]:
quantile_low = df.quantile(0.1)
quantile_high = df.quantile(0.9)
iqr = quantile_high - quantile_low

In [5]:
outliers = (df < quantile_low) | (df > quantile_high)
df_new = df[outliers.sum(axis=1) <= 12]

Compute 7 scorse and sort images by order in survey

In [7]:
vgg_df=pd.read_csv('vgg_scores_wordnet_id_closest_label.csv',index_col=0)
vgg_df['typicality_score'] = np.round(vgg_df['closest_prob'] * 100,2)
vgg_df['7_score']= np.round(vgg_df['typicality_score']*7/100,2)
vgg_df['type'] = vgg_df['type'].apply(lambda x: x.lower())

In [8]:
vgg_df['number'] = vgg_df['dir'].apply(lambda x: int(x.split('\\')[-1].split('.')[0]))
vgg_df = vgg_df.sort_values(['type','number']).reset_index(drop=True)

# Correlation with human ratings

Changes from original script: 
1. Use spearman rank correlation instead of pearson
2. Compute average human ratings first, and then compute correlation with neural net typicality scores

without dropping outliers

In [9]:
final_corr = dict()
for t in np.unique(vgg_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    human_ratings = df[col_names] 
    net_ratings =vgg_df[vgg_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
#     net_scores = net_ratings['typicality_score'].values
    
    coef, p=stats.spearmanr(human_scores, net_scores)
#     coef, p=stats.pearsonr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.19573220898699903
beach:  -0.19999999999999998
car:  -0.15882352941176472
church:  0.07064019572463123
lighthouse:  0.5529411764705882
mountain:  0.5342164801675237
mug:  -0.04274135279917027
plane:  0.469462967419945


In [10]:
print(np.mean(list(final_corr.values())))

0.17767851831984402


after dropping outliers

In [11]:
final_corr = dict()
for t in np.unique(vgg_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    
    human_ratings = df_new[col_names] 
    net_ratings = vgg_df[vgg_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
    coef, p=stats.spearmanr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.16924213559026233
beach:  -0.23416814791772844
car:  -0.1723124107319134
church:  0.06916852498036807
lighthouse:  0.5567016346723356
mountain:  0.44775395197848766
mug:  0.029520367555897542
plane:  0.3716830330568026


In [12]:
print(np.mean(list(final_corr.values())))

0.15469863614806398
